In [60]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau

In [61]:
# Define the directory containing the Eurosat dataset
data_dir = r"D:\MCA 3rd SEM\Small Industrial Project\DATASET_COMPARE\90"

In [62]:
# Define the input shape for the model
input_shape = (64, 64, 3)

In [63]:
# Use ReduceLROnPlateau callback
lr_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                 patience=3, 
                                 verbose=1, 
                                 factor=0.5, 
                                 min_lr=0.00001)

In [64]:
# Create an image data generator with more augmentations and validation split
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # use preprocess_input from resnet50
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # add this line for validation split
)

In [65]:
# Create a train generator with only `stage` images per class
train_generator = datagen.flow_from_directory(
    data_dir, 
    target_size=(64,64), 
    batch_size=32, 
    subset='training'  # specify this as 'training'
)

Found 720 images belonging to 10 classes.


In [66]:
# Create a validation generator
validation_generator = datagen.flow_from_directory(
    data_dir, 
    target_size=(64,64), 
    batch_size=32, 
    subset='validation'  # specify this as 'validation'
)

Found 180 images belonging to 10 classes.


In [67]:
# Create a new instance of the model for transfer learning
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dropout(0.5)) # Add dropout layer to reduce overfitting

In [68]:
# The number of classes is automatically inferred from the subdirectories
model.add(Dense(train_generator.num_classes, activation='softmax'))

In [69]:
# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [70]:
# Train the model with validation data and callbacks
history = model.fit(
    train_generator, 
    epochs=10, 
    validation_data=validation_generator,  # use validation_data instead of validation_split
    callbacks=[lr_reduction]
)

Epoch 1/10
23/23 [==============================] - 41s 1s/step - loss: 2.2777 - accuracy: 0.5194 - val_loss: 19167.4160 - val_accuracy: 0.1000 - lr: 0.0010
Epoch 2/10
23/23 [==============================] - 28s 1s/step - loss: 2.2261 - accuracy: 0.6347 - val_loss: 10329106.0000 - val_accuracy: 0.1000 - lr: 0.0010
Epoch 3/10
23/23 [==============================] - 29s 1s/step - loss: 1.9741 - accuracy: 0.7069 - val_loss: 78427.0859 - val_accuracy: 0.1611 - lr: 0.0010
Epoch 4/10
23/23 [==============================] - ETA: 0s - loss: 1.8250 - accuracy: 0.7194
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
23/23 [==============================] - 29s 1s/step - loss: 1.8250 - accuracy: 0.7194 - val_loss: 67156.8906 - val_accuracy: 0.1000 - lr: 0.0010
Epoch 5/10
23/23 [==============================] - 29s 1s/step - loss: 2.3840 - accuracy: 0.7222 - val_loss: 34.4176 - val_accuracy: 0.1167 - lr: 5.0000e-04
Epoch 6/10
23/23 [==============================] - 

In [71]:
# Define the path to your directory for testing
dir_path = r"D:\MCA 3rd SEM\Small Industrial Project\TEST"

In [72]:
# Get a list of all image files in the directory
img_files = [f for f in os.listdir(dir_path) if f.endswith('.jpg')]

# Loop over each image file
for img_file in img_files:
    # Load and preprocess the image
    img = load_img(os.path.join(dir_path, img_file), target_size=input_shape[:2])
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    # Use the trained model to make a prediction
    preds = model.predict(x)

    # Decode the prediction
    predicted_class = np.argmax(preds[0])

    print(f'The predicted class for {img_file} is: {classes[predicted_class]}')

1/1 [==============================] - 1s 812ms/step
The predicted class for AnnualCrop_1.jpg is: HerbaceousVegetation
1/1 [==============================] - 0s 41ms/step
The predicted class for Forest_1.jpg is: Forest
1/1 [==============================] - 0s 36ms/step
The predicted class for HerbaceousVegetation_1.jpg is: HerbaceousVegetation
1/1 [==============================] - 0s 37ms/step
The predicted class for Highway_1.jpg is: Industrial
1/1 [==============================] - 0s 39ms/step
The predicted class for Industrial_1.jpg is: Industrial
1/1 [==============================] - 0s 36ms/step
The predicted class for Pasture_1.jpg is: Pasture
1/1 [==============================] - 0s 40ms/step
The predicted class for PermanentCrop_1.jpg is: PermanentCrop
1/1 [==============================] - 0s 38ms/step
The predicted class for Residential_1.jpg is: Residential
1/1 [==============================] - 0s 36ms/step
The predicted class for River_1.jpg is: River
1/1 [===========

In [78]:
from keras.applications.resnet50 import preprocess_input

# Define the path to your image for testing
# img_path = r"D:\MCA 3rd SEM\Small Industrial Project\DATASET_COMPARE\90\AnnualCrop\AnnualCrop_4.jpg"
img_path = r'D:\MCA 3rd SEM\Small Industrial Project\DATASET_COMPARE\90\HerbaceousVegetation\HerbaceousVegetation_1.jpg'
# img_path = r"D:\MCA 3rd SEM\Small Industrial Project\DATASET_COMPARE\90\Industrial\Industrial_90.jpg"
# img_path = r"D:\MCA 3rd SEM\Small Industrial Project\DATASET_COMPARE\90\SeaLake\SeaLake_.jpg"

# Load and preprocess the image
img = load_img(img_path, target_size=input_shape[:2])
x = img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

# Use the trained model to make a prediction
preds = model.predict(x)

# Get the class labels from the train generator
classes = list(train_generator.class_indices.keys())

# Decode the prediction
predicted_class = np.argmax(preds[0])

print(f'The predicted class is: {classes[predicted_class]}')

1/1 [==============================] - 0s 411ms/step
The predicted class is: HerbaceousVegetation
